# Begriffe 1: Der Bias

### Integrationskonstante

Bereits bei der Linearen Regression hatten wir festgestellt, dass wir neben dem Parameter der Steigung $m$ noch einen zweiten Parameter $b$ benötigten, obwohl es sich eigentlich um ein eindimensionales Problem handelt. Auch bei der  Polynomialen Regression tauchte eine Konstante $a_0$ auf. Diese Konstanten waren notwendig, denn die übrigen Parameter sorgten nur dafür, dass die Ausgleichsgerade bzw. das Ausgleichspolynom die *Änderungsrate zwischen den Messwerten* berücksichtigt, aber nicht deren absolute Lage; d.h. die approximierende Kurve hat zwar die richtige Gestalt, aber nicht die richtige Position und muss noch vertikal verschoben werden. Dazu ist ein konstanter Faktor notwendig, den wir in der Schulmathematik als **Integrationskonstante** kennengelernt haben. Ohne diese Konstante könnten wir nur Funktionen approximieren, die im Nullpunkt den Wert Null haben.

Auch im Deep Learning taucht diese Konstante auf; sie wird dann der **Bias** genannt.

### Der Bias

Der Bias ist für neuronale Netze unerlässlich; nur auf diese Weise lässt sich bei Eingangssignalen 0 ein von 0 verschiedenes Ausgangssignal erzielen. Der Bias entspricht bei der Linearen und Polynomialen Regression dem konstanten Koeffizienten $a_0$. 

Bereits beim Perzeptron haben wir den Bias $b$ kennengelernt. Dort wurde er neben den beiden Gewichten $w_1,w_2$ eingeführt, um einen von 0 verschiedenen Ausgang zu erzielen, obwohl beide Eingänge 0 sind. Dies führte zu der Lernregel
$$
\begin{equation}
\begin{split}
w_{1,2} &\rightarrow w_{1,2} + \alpha (y - o) x_{1,2}\\
b &\rightarrow b + \alpha (y - o )
\end{split}
\end{equation}
$$

Wir brauchten also zwei Anweisungen, um die Gewichte und den Bias anzupassen. 

Es ist aber lästig, ständig zwei Regeln zu entwickeln, für $N$ Gewichte und den Bias. Mit einem einfachen Trick lässt sich das problem lösen.

### Der verschwundene Bias

Wir zeigen das Vorgehen am Beispiel des Perzeptrons. Wir führen ein weiteres ("virtuelles") Eingangsneuron ein, an dem immer der Wert $1$ anliegt. Dieses Neuron bekommt den Index 0. Damit haben wir nun die Eingangsneuronen $x_0,x_1,x_2$. Betrachten wir nun $b$ als Gewicht dieses fiktiven Eingangsneurons und bezeichnen es mit $w_0$, so erhalten wir folgendes KNN:

![Das Perzeptron mit virtuellem Input](images/Perzeptron2.png)

Unsere Verlustfunktion lässt sich nun einfach schreiben als

$$L(w_k) = \sum_{k=0}^2 (w_k x_k - y_k)^2$$

und unser Lernalgorithmus wird einfach zu

$$w_k \rightarrow w_k + \alpha (y - o) x_k, \;\;\;k=0,1,2.$$

Dieser kleine Kniff macht die Formeln sehr übersichtlich. Man muss sich lediglich merken, dass das 0. Neuron immer den Wert 1 hat. Wir werden dies in Zukunft bei Bedarf verwenden, wie es auch in der Fachliteratur häufig gemacht wird.

### Implementierung des Perzeptrons, 2.Version

Wir wollen den Trick mit dem verschwundenen Bias direkt einmal bei der Implementierung des Perzeptrons ausprobieren.

In [1]:
import numpy as np
import matplotlib as plt
%matplotlib inline

#### Die Funktionen vereinfachen sich noch einmal

In [2]:
def output(x,w):
    return 1 if np.sum(w * x) > 0 else 0

def lernschritt(x,y,w,alpha): 
    return w + alpha * (y - output(x,w)) * x

def training(testdaten,w,alpha=0.01,maxiter=1000):
    for i in np.random.randint(4,size=maxiter):
        w = lernschritt(testdaten[i,:-1],testdaten[i,-1],w,alpha)              
    return w

#### Training und Test

##### Anpassung der Testdaten

In [3]:
test_und   = np.array([[0,0,0],[0,1,0],[1,0,0],[1,1,1]])
test_nund  = np.array([[0,0,1],[0,1,1],[1,0,1],[1,1,0]])
test_oder  = np.array([[0,0,0],[0,1,1],[1,0,1],[1,1,1]])
test_xoder = np.array([[0,0,0],[0,1,1],[1,0,1],[1,1,0]])

ones = np.array([[1] for _ in range(len(test_nund))])

test_und   = np.append(ones,test_und,axis=1)
test_nund  = np.append(ones,test_nund,axis=1)
test_oder  = np.append(ones,test_oder,axis=1)
test_xoder = np.append(ones,test_xoder,axis=1)

In [6]:
# Trainiert und testet ein Perzeptron

def tut(name,daten):

    w = training(daten,np.zeros(len(daten) - 1))
    
    print(f'{name}: ')
    print(f'Gewichte = {w}\n')
    for x in daten:
        o = output(x[:-1],w)
        check = '\u2713' if o == x[3] else '\u21af'
        print(f'{x[1]},{x[2]} -> {x[3]}, Output: {o}   {check}')
    print('\n')

In [5]:
tut('UND-Verknüpfung',test_und)
tut('NICHT-UND-Verknüpfung',test_nund)
tut('ODER-Verknüpfung',test_oder)
tut('XODER-Verknüpfung',test_xoder)

UND-Verknüpfung: 
Gewichte = [-0.01  0.01  0.01]

0,0 -> 0, Output: 0   ✓
0,1 -> 0, Output: 0   ✓
1,0 -> 0, Output: 0   ✓
1,1 -> 1, Output: 1   ✓


NICHT-UND-Verknüpfung: 
Gewichte = [ 0.03 -0.02 -0.01]

0,0 -> 1, Output: 1   ✓
0,1 -> 1, Output: 1   ✓
1,0 -> 1, Output: 1   ✓
1,1 -> 0, Output: 0   ✓


ODER-Verknüpfung: 
Gewichte = [0.   0.01 0.01]

0,0 -> 0, Output: 0   ✓
0,1 -> 1, Output: 1   ✓
1,0 -> 1, Output: 1   ✓
1,1 -> 1, Output: 1   ✓


XODER-Verknüpfung: 
Gewichte = [0.01 0.   0.  ]

0,0 -> 0, Output: 1   ↯
0,1 -> 1, Output: 1   ✓
1,0 -> 1, Output: 1   ✓
1,1 -> 0, Output: 1   ↯




### Zusammenfassung

1. Der Bias ist im Deep Learning ein wichtiges Element, da durch ihn einige Aufgaben erst gelöst werden können.
2. Indem der Bias als Gewicht eines zusätzlichen fiktiven Eingabeneurons betrachtet wird, vereinfachen sich viele Rechnungen. Allerdings müssen die Testdaten angepasst werden.

### Links

[1] [Bias in der Wikipedia](https://de.wikipedia.org/wiki/K%C3%BCnstliches_Neuron#Bias)